In [1]:
import numpy as np
import cv2
import os
#import scipy.io as sio
#from matplotlib import pyplot as plt
import re

#import imageio

In [2]:
def load_images_from_folder(folder):
    images = []
    fs = []
    for filename in sorted(os.listdir(folder)):
        f = os.path.join(folder,filename)
        img =cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        #print (f)
        if img is not None:
            images.append(img)
            fs.append((f[-7:-4]))
    return images, fs

In [3]:
drawing = False
ix, iy = -1, -1

def calc_radius(x, y):
    global ix, iy
    diff1 = ix - x
    diff2 = iy - y

    r = sqrt(diff1*diff1 + diff2*diff2)
    return int(r)

def draw(event, x, y, flags, param):
    global drawing, ix, iy, shape, canvas, brush

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            if shape == 1:
                cv2.circle(canvas, (x, y), pencil, color, -1)
            elif shape == 2:
                cv2.circle(canvas, (x, y), brush, color, -1)
            elif shape == 3:
                cv2.circle(canvas, (x, y), eraser, (255, 255, 255), -1)
            elif shape == 5:
                cv2.rectangle(canvas, (ix, iy), (x, y), color, -1)
            elif shape == 6:
                cv2.circle(canvas, (x, y), calc_radius(x, y), color, -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        if shape == 1:
            cv2.circle(canvas, (x, y), pencil, color, -1)
        elif shape == 2:
            cv2.circle(canvas, (x, y), brush, color, -1)
        elif shape == 3:
            cv2.circle(canvas, (x, y), eraser, (255, 255, 255), -1)
        elif shape == 4:
            cv2.line(canvas, (ix, iy), (x, y), color, pencil)
        elif shape == 5:
            cv2.rectangle(canvas, (ix, iy), (x, y), color, -1)
        elif shape == 6:
            cv2.circle(canvas, (x, y), calc_radius(x, y), color, -1)

def display_shape():
    global shape
    if shape == 0:
        cv2.putText(obj, 'Off', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 1:
        cv2.putText(obj, 'Pencil', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 2:
        cv2.putText(obj, 'Brush', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 3:
        cv2.putText(obj, 'Eraser', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 4:
        cv2.putText(obj, 'Line', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 5:
        cv2.putText(obj, 'Rectangle', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)
    elif shape == 6:
        cv2.putText(obj, 'Circle', (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 2)


def do_nothing(param):
    pass

# read data

In [4]:
data, fs = load_images_from_folder(r'..') # change to the folder where the image is stored
np.shape(data)
data = np.swapaxes(data,0, 2) # change the axis of the images
print (np.shape(data))
filename = list(map(lambda x:re.sub("\D", "", x), fs)) # note that it is not in the right order 1 is followed by 10

(256, 256, 256)


# label the image and save it in a new folder

In [5]:
i = 17 # change i to the image you want to see
path = r'..'# change to the folder to save the annoation data

canvas = data[:,:,i]
canvas = cv2.cvtColor(canvas, cv2.COLOR_GRAY2RGB) # THERE IS A bug, cannot change mode

palette = np.zeros((300, 512, 3), np.uint8)
obj = np.zeros((300, 512, 3), np.uint8)


cv2.namedWindow('Canvas')
cv2.namedWindow('Palette')
cv2.namedWindow('Object')



cv2.createTrackbar('R', 'Palette', 0, 255, do_nothing)
cv2.createTrackbar('B', 'Palette', 0, 255, do_nothing)
cv2.createTrackbar('G', 'Palette', 0, 255, do_nothing)

cv2.createTrackbar('Shape', 'Object', 0, 6, do_nothing)
cv2.createTrackbar('Pencil', 'Object', 0, 5, do_nothing)
cv2.createTrackbar('Brush', 'Object', 0, 12, do_nothing)
cv2.createTrackbar('Eraser', 'Object', 0, 12, do_nothing)

cv2.setMouseCallback('Canvas', draw)

while True:
        global shape, color, brush, eraser, pencil
        cv2.imshow('Canvas', canvas)
        cv2.imshow('Palette', palette)
        cv2.imshow('Object', obj)

        k = cv2.waitKey(1) & 0xFF

        if k == ord('q'):
                break
        elif k == ord('s'):
                print('saving image')
                cv2.imwrite(path+'\\'+filename[i]+'.jpg',np.fliplr(np.rot90(canvas, k =-1)) ) # use the same file name to save images
        elif k == ord('o'):
                canvas = cv2.imread('canvas.jpg')

        r = cv2.getTrackbarPos('R', 'Palette')
        b = cv2.getTrackbarPos('B', 'Palette')
        g = cv2.getTrackbarPos('G', 'Palette')

        color = [b, g, r]
        palette[:] = color

        brush = cv2.getTrackbarPos('Brush', 'Object')
        eraser = cv2.getTrackbarPos('Eraser', 'Object')
        pencil = cv2.getTrackbarPos('Pencil', 'Object')

        shape = cv2.getTrackbarPos('Shape', 'Object')

        obj[:] = [0, 0, 0]
        display_shape()

cv2.destroyAllWindows()


saving image
